In [52]:
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge , ElasticNet, LogisticRegression
from sklearn.model_selection import KFold , StratifiedKFold
from sklearn.model_selection import cross_val_predict,cross_val_score
import os
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline

In [54]:
df = pd.read_csv('Glass.csv')
df

RI     Na    Mg    Al     Si     K    Ca    Ba   Fe  \
0    1.52101  13.64  4.49  1.10  71.78  0.06  8.75  0.00  0.0   
1    1.51761  13.89  3.60  1.36  72.73  0.48  7.83  0.00  0.0   
2    1.51618  13.53  3.55  1.54  72.99  0.39  7.78  0.00  0.0   
3    1.51766  13.21  3.69  1.29  72.61  0.57  8.22  0.00  0.0   
4    1.51742  13.27  3.62  1.24  73.08  0.55  8.07  0.00  0.0   
..       ...    ...   ...   ...    ...   ...   ...   ...  ...   
209  1.51623  14.14  0.00  2.88  72.61  0.08  9.18  1.06  0.0   
210  1.51685  14.92  0.00  1.99  73.06  0.00  8.40  1.59  0.0   
211  1.52065  14.36  0.00  2.02  73.42  0.00  8.44  1.64  0.0   
212  1.51651  14.38  0.00  1.94  73.61  0.00  8.48  1.57  0.0   
213  1.51711  14.23  0.00  2.08  73.36  0.00  8.62  1.67  0.0   

                                 Type  
0    building_windows_float_processed  
1    building_windows_float_processed  
2    building_windows_float_processed  
3    building_windows_float_processed  
4    building_windows_float_processed  
..                                ...  
209                         headlamps  
210                         headlamps  
211                         headlamps  
212                         headlamps  
213                         headlamps  

[214 rows x 10 columns]

In [56]:
X , y = df.drop('Type', axis=1) , df['Type']


In [58]:
# Pipe 
lr = LogisticRegression()
scl_std , scl_min = StandardScaler() , MinMaxScaler()
kfold = StratifiedKFold(n_splits=5,shuffle=True,random_state=24)

pipe = Pipeline([('scaler', None),('LR', lr)])
pipe.get_params()



{'memory': None,
 'steps': [('scaler', None), ('LR', LogisticRegression())],
 'transform_input': None,
 'verbose': False,
 'scaler': None,
 'LR': LogisticRegression(),
 'LR__C': 1.0,
 'LR__class_weight': None,
 'LR__dual': False,
 'LR__fit_intercept': True,
 'LR__intercept_scaling': 1,
 'LR__l1_ratio': None,
 'LR__max_iter': 100,
 'LR__multi_class': 'deprecated',
 'LR__n_jobs': None,
 'LR__penalty': 'l2',
 'LR__random_state': None,
 'LR__solver': 'lbfgs',
 'LR__tol': 0.0001,
 'LR__verbose': 0,
 'LR__warm_start': False}

In [70]:

params = {'LR__penalty':['l2' , 'l1' , 'elasticnet' , None] , 'LR__l1_ratio' : np.linspace(0.001 ,1 ,5) , 'LR__multi_class':['ovr' , 'multinomial'],
          'scaler' : [scl_std , scl_min , None]}
gsv = GridSearchCV(pipe , param_grid=params , cv = kfold , scoring='accuracy')
gsv.fit(X,y)

C:\Users\dbda.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1196: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
C:\Users\dbda.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
C:\Users\dbda.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1196: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
C:\Users\dbda.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) inste

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=24, shuffle=True),
             estimator=Pipeline(steps=[('scaler', None),
                                       ('LR', LogisticRegression())]),
             param_grid={'LR__l1_ratio': array([0.001  , 0.25075, 0.5005 , 0.75025, 1.     ]),
                         'LR__multi_class': ['ovr', 'multinomial'],
                         'LR__penalty': ['l2', 'l1', 'elasticnet', None],
                         'scaler': [StandardScaler(), MinMaxScaler(), None]},
             scoring='accuracy')

In [71]:
gsv.best_score_

0.6499446290143964

In [72]:
gsv.best_params_

{'LR__l1_ratio': 0.001,
 'LR__multi_class': 'ovr',
 'LR__penalty': None,
 'scaler': StandardScaler()}

In [73]:
pd.DataFrame(gsv.cv_results_)

mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0         0.012781  1.723229e-03         0.001585        0.000480   
1         0.011170  7.459197e-04         0.001396        0.000488   
2         0.053256  2.982107e-03         0.001595        0.000488   
3         0.002194  3.985686e-04         0.000000        0.000000   
4         0.002190  3.851847e-04         0.000000        0.000000   
..             ...           ...              ...             ...   
115       0.001995  2.336015e-07         0.000000        0.000000   
116       0.000598  4.886166e-04         0.000000        0.000000   
117       0.016355  4.886361e-04         0.001795        0.000399   
118       0.015757  3.990650e-04         0.001596        0.000489   
119       0.015159  3.989220e-04         0.001795        0.000399   

     param_LR__l1_ratio param_LR__multi_class param_LR__penalty  \
0                 0.001                   ovr                l2   
1                 0.001                   ovr                l2   
2                 0.001                   ovr                l2   
3                 0.001                   ovr                l1   
4                 0.001                   ovr                l1   
..                  ...                   ...               ...   
115               1.000           multinomial        elasticnet   
116               1.000           multinomial        elasticnet   
117               1.000           multinomial              None   
118               1.000           multinomial              None   
119               1.000           multinomial              None   

         param_scaler                                             params  \
0    StandardScaler()  {'LR__l1_ratio': 0.001, 'LR__multi_class': 'ov...   
1      MinMaxScaler()  {'LR__l1_ratio': 0.001, 'LR__multi_class': 'ov...   
2                None  {'LR__l1_ratio': 0.001, 'LR__multi_class': 'ov...   
3    StandardScaler()  {'LR__l1_ratio': 0.001, 'LR__multi_class': 'ov...   
4      MinMaxScaler()  {'LR__l1_ratio': 0.001, 'LR__multi_class': 'ov...   
..                ...                                                ...   
115    MinMaxScaler()  {'LR__l1_ratio': 1.0, 'LR__multi_class': 'mult...   
116              None  {'LR__l1_ratio': 1.0, 'LR__multi_class': 'mult...   
117  StandardScaler()  {'LR__l1_ratio': 1.0, 'LR__multi_class': 'mult...   
118    MinMaxScaler()  {'LR__l1_ratio': 1.0, 'LR__multi_class': 'mult...   
119              None  {'LR__l1_ratio': 1.0, 'LR__multi_class': 'mult...   

     split0_test_score  split1_test_score  split2_test_score  \
0             0.604651           0.627907           0.511628   
1             0.558140           0.604651           0.534884   
2             0.604651           0.627907           0.604651   
3                  NaN                NaN                NaN   
4                  NaN                NaN                NaN   
..                 ...                ...                ...   
115                NaN                NaN                NaN   
116                NaN                NaN                NaN   
117           0.674419           0.627907           0.604651   
118           0.674419           0.674419           0.558140   
119           0.651163           0.674419           0.651163   

     split3_test_score  split4_test_score  mean_test_score  std_test_score  \
0             0.581395           0.738095         0.612735        0.073777   
1             0.465116           0.666667         0.565891        0.067580   
2             0.534884           0.714286         0.617276        0.057674   
3                  NaN                NaN              NaN             NaN   
4                  NaN                NaN              NaN             NaN   
..                 ...                ...              ...             ...   
115                NaN                NaN              NaN             NaN   
116                NaN                NaN              NaN       

#### In case of classification data inplace of kfold use Stratifykfold to get better results